# News Categorization

- 비슷한 뉴스를 어떻게 선정할까?
- 뉴스들이 있다면 컴퓨터는 문자를 그대로 이해하지 못하기 때문에 문자를 숫자로 바꾸어야 한다.
- 숫자로 '유사하다'는 어떻게 표현할까? '유사하다 = 가깝다'로 표현할 수 있다. 좌표평면에서 기본적으로 쓰이는 피타고라스의 정리를 통해 두 점이 가까운지 먼지를 알 수 있다. 
- 그래서 궁극적으로 하고 싶은 것은 문자를 숫자로, 숫자는 vector로 표현하고 싶은 것이다.

## 문자 -> 숫자 -> Vector

### 문자를 Vector로 - One-hot Encoding

- 하나의 단어를 Vector의 Indox로 인식, 단어 존재시 1 없으면 0

In [1]:
Rome = [1,0,0,0,0,0,0,0,0,0]
Paris = [0,1,0,0,0,0,0,0,0,0]
Italy = [0,0,1,0,0,0,0,0,0,0]
France = [0,0,0,1,0,0,0,0,0,0]

### Bag of words

- 단어별로 인덱스를 부여해서, 한 문장(또는 문서)의 단어의 개수를 Vector로 표현

<img src="../img/Screen Shot 2018-08-15 at 7.17.55 PM.png" width="500">

## 유사성

### Distance measure

- 고등학교 때 배운 2차원 평면상 거리측정 방법들

<img src="../img/Screen Shot 2018-08-15 at 7.20.56 PM.png" width="600">

### Euclidian distance

- 피타고라스의 정리, 두 점 사이의 직선의 거리

<img src="../img/Screen Shot 2018-08-15 at 7.23.17 PM.png" width="600">

### Cosine distance

- 두 점 사이의 각도

<img src="../img/Screen Shot 2018-08-15 at 7.23.27 PM.png" width="600">

일반적으로 Euclidian distance보다는 Cosine distance를 더 많이 사용한다.

## Code

### Data set

- 축구와 야구 선수들의 영문 기사를 분류해보자!
- 1,2,3,4 야구 / 5,6,7,8 축구 (news_data 폴더 참고)

### Process

- 파일을 불러오기
- 파일을 읽어서 단어사전 (corpus) 만들기
- 단어별로 Index 만들기
- 만들어진 인덱스로 문서별로 Bag of words vector 생성 - 비교하고자 하는 문서 비교하기
- 얼마나 맞는지 측정하기

### 파일 불러오기

In [2]:
import os

def get_file_list(dir_name):
    return os.listdir(dir_name)

print(len(get_file_list('news_data')))

if __name__ == "__main__":
    dir_name = "news_data"
    
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) 
                 for file_name in file_list]
    print(len(file_list))

80
80


- os.listdir

os.listdir을 이용하면 해당 디렉터리에 있는 파일들의 리스트를 가져오게 된다. 여기서 구해지는 파일 리스트는 파일명만 포함되어 있으므로 경로를 포함한 파일명을 구하기 위해서는 입력으로 받은 dirname을 앞에 덧붙여 주어야 한다.

- os.path.join

디렉터리와 파일들을 연결하려고 할 때 사용한다. 보통 /를 통해 join을 해주지만 python에서는 os.path.join을 통해 join을 해주어야 한다. os 모듈에 있는 os.path.join로 디렉터리를 포함한 전체 경로를 쉽게 가져올 수 있다.

쉽게 말해 os.listdir을 통해 전체 파일들의 리스트를 가져와서, os.path.join을 통해 전체 경로를 연결하여 다시 전체 파일들의 리스트를 가져온다.

### 파일별로 내용읽기

In [3]:
def get_conetents(file_list):
    y_class = []
    X_text = []

    # baseball - 0, soccer - 1
    class_dict = {
        1: "0", 2: "0", 3:"0", 4:"0", 5:"1", 6:"1", 7:"1", 8:"1"}

    for file_name in file_list:
        try:
            # cp949 - windows file (utf - mac, linux file)
            f = open(file_name, "r",  encoding="cp949")
            
            # os.sep[1] - file name only, split("_")[0] - before underbar
            # os.sep - Get directory separator
            # ex. 1 in 1_Dae-Ho Lee walk-off homer gives Mariners ...
            category = int(file_name.split(os.sep)[1].split("_")[0])
            
            # print(file_name.split(os.sep))
            # ex. ['news_data', '3_SEUNG-HWAN OH SHUTS DOWN TWINS IN FIRST SPRING ACTION.txt']
            
            # y_class is expressed as 0 class(baseball) and 1 class(soccer)
            y_class.append(class_dict[category])
            
            # X_test is 1-dimensional list
            X_text.append(f.read())
            f.close()
        except UnicodeDecodeError as e:
            print(e)
            print(file_name)
    return X_text, y_class

# test
X_text, y_class = get_conetents(file_list)
# print(y_class)
# print(X_text)

### Corpus 만들기 + 단어별 index 생성하기

- Corpus란, text안에 있는 word들에 대한 index를 만들어주는 것. 각각의 word가 몇 번째 index에 속한다는 것을 선언해주는 것.

In [4]:
# additional section
def get_cleaned_text(text):
    '''regular expression - eliminate meaningless sentence protection'''
    import re
    text = re.sub('\W+','', text.lower() )
    return text

print(get_cleaned_text("I'm Yours"))

imyours


In [5]:
def get_corpus_dict(text):
    # 2-dimensional list
    text = [sentence.split() for sentence in text]
    
    # 1-dimensioal list
    cleaned_words = [get_cleaned_text(word) 
                    for words in text for word in words]

    corpus_dict = dict()
    for i, v in enumerate(set(cleaned_words)):
        corpus_dict[v] = i
    return corpus_dict

corpus = get_corpus_dict(X_text)
print("Number of words : {}".format(len(corpus)))
# print('corpus', corpus)

Number of words : 4024


위의 코드에서 set은 다음과 같은 2가지 큰 특징이 있다.

- 중복을 허용하지 않는다.
- 순서가 없다(Unordered).

또한 리스트나 튜플은 순서가 있기(ordered) 때문에 인덱싱을 통해 자료형의 값을 얻을 수 있지만 set 자료형은 순서가 없기(unordered) 때문에 인덱싱으로 값을 얻을 수 없다. 만약 set 자료형에 저장된 값을 인덱싱으로 접근하려면 다음과 같이 리스트나 튜플로 변환한 후 해야 한다.

위에서 set을 쓴 이유도 중복을 없애기 위해서 쓴 것이다. 단어들마다 index를 만들어주기 위해서 중복되는 단어들을 set을 통해 없앤 뒤에 for문을 통해 index를 만들어준다. 

### 문서별로 Bag of words vector 생성

In [6]:
def get_count_vector(text, corpus):
    text = [sentence.split() for sentence in text]
    
    # 2-dimensional list
    # The number that the words in the documents correspond to the index number in the corpus.
    word_number_list = [[corpus[get_cleaned_text(word)] 
                         for word in words] for words in text]
    # print('corpus', corpus)
#     print('word_number_list', word_number_list)
    
    # 80 x 4024 matrices
    X_vector = [[0 for _ in range(len(corpus))] 
                for _ in range(len(text))]
#     print(X_vector[0])

    # Check the number of words in one text
    for i, text in enumerate(word_number_list):
        for word_number in text:
            X_vector[i][word_number] += 1
    return X_vector

# test
corpus = get_corpus_dict(X_text)
# print(get_count_vector(X_text, corpus))

### 비교하기

<img src="../img/Screen Shot 2018-08-15 at 10.18.42 PM.png" width="500">

In [7]:
import math
def get_cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

여기서 v1, v2는 문서를 의미한다. 그러니까 얼마의 similarity score를 가지는 지를 표현하는 코드이다.

### 비교결과 정리하기

In [10]:
def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(
            get_cosine_similarity(source_vector, target_vector))
    return similarity_list


def get_top_n_similarity_news(similarity_score, n):
    import operator
    total_score = {i:v for i, v in enumerate(similarity_score)}
#     print("total_score\n" + str(total_score))
    sorted_total_score = sorted(total_score.items(), key=operator.itemgetter(1))
    return list(reversed(sorted_total_score))[1:n+1]

In [9]:
# test
X_vector = get_count_vector(X_text, corpus)
source_number = 1
test_similarity_score = get_similarity_score(X_vector, source_number)
test_similarity_news = get_top_n_similarity_news(test_similarity_score, 5)

total_score
{0: 0.422127681059035, 1: 1.0, 2: 0.560082394374579, 3: 0.2968763938415094, 4: 0.5514983941504927, 5: 0.6076835623757505, 6: 0.5060457507814917, 7: 0.6314062762993071, 8: 0.5421767411201769, 9: 0.427426276966268, 10: 0.5912746373712418, 11: 0.6029946996569844, 12: 0.7144117698730763, 13: 0.5189551148833238, 14: 0.5475429873588856, 15: 0.5643477968473063, 16: 0.47443968388194596, 17: 0.6009817862519662, 18: 0.3522531739514477, 19: 0.6582264823046564, 20: 0.47604248440179164, 21: 0.5306292591824382, 22: 0.4914523616067112, 23: 0.49193602225302246, 24: 0.5696030889639794, 25: 0.6232503314246413, 26: 0.5926674229884631, 27: 0.3687409228350686, 28: 0.671978067683249, 29: 0.6348050371984161, 30: 0.5770610597506486, 31: 0.542277551828845, 32: 0.4257576283238868, 33: 0.6063203769697062, 34: 0.5062393559988809, 35: 0.34700237194342054, 36: 0.344447481913586, 37: 0.6038924815196922, 38: 0.5142741232363717, 39: 0.6732690733648422, 40: 0.4847200928510529, 41: 0.5808629216843902, 42: 0.

### 성능 측정하기

In [11]:
def get_accuracy(similarity_list, y_class, source_news):
    source_class = y_class[source_news]
#     print(similarity_list)
#     print(sum([source_class == y_class[i[0]]
#                 for i in similarity_list]))
    return sum([source_class == y_class[i[0]] 
                for i in similarity_list]) / len(similarity_list)

In [12]:
if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) 
                 for file_name in file_list]

    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    print("Number of words : {0}".format(len(corpus)))
    X_vector = get_count_vector(X_text, corpus)

    result = []

    for i in range(80):
        source_number = i
        
        # 80 x 80
        similarity_score = get_similarity_score(X_vector, source_number)
        
        similarity_news = get_top_n_similarity_news(similarity_score, 5)
        
        accuracy_score = get_accuracy(similarity_news, y_class, source_number)
        result.append(accuracy_score)
    print(round((sum(result) / 80), 2))

Number of words : 4024
0.75
